In [1]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

import tensorflow as tf 
import tensorboard as tb 
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss,MAE,MAPE,RMSE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from pytorch_lightning.callbacks import ModelCheckpoint

In [2]:
import pandas as pd

# Load the excel file
data = pd.read_excel('./dataset/Jiuzhaigou.xlsx')
data["year"] = data["year"].astype(str)
data["day"] = data["day"].astype(str)
data["tourist"] = data["tourist"].astype("float64")
data["pc_Jiuzhaigou"] = data["pc_Jiuzhaigou"].astype("float64")
data["mob_Jiuzhaigou"] = data["mob_Jiuzhaigou"].astype("float64")
data["pc_SichuanEpidemic"] = data["pc_SichuanEpidemic"].astype("float64")
data["mob_SichuanEpidemic"] = data["mob_SichuanEpidemic"].astype("float64")
data["month"] = data["month"].astype(str)
data

,date,tourist,time_idx,pc_Jiuzhaigou,mob_Jiuzhaigou,pc_SichuanEpidemic,mob_SichuanEpidemic,weekday,year,month,day,destination,Trend,Seasonal,Resid
0,2020-04-01,208.0,1,388.0,856.0,271.0,959.0,Tuesday,2020,4,1,JiuZhaigou,311.003039,1931.261309,3965.735652
1,2020-04-02,295.0,2,445.0,873.0,243.0,933.0,Wednesday,2020,4,2,JiuZhaigou,341.195455,1946.020432,4007.784113
2,2020-04-03,311.0,3,333.0,877.0,201.0,841.0,Thursday,2020,4,3,JiuZhaigou,372.267806,1847.161177,4091.571017
3,2020-04-04,528.0,4,218.0,945.0,116.0,886.0,Friday,2020,4,4,JiuZhaigou,404.858892,2127.398485,3995.742622
4,2020-04-05,2018.0,5,180.0,912.0,106.0,794.0,Saturday,2020,4,5,JiuZhaigou,439.053772,2039.800262,5539.145966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,2021-09-08,3728.0,526,454.0,965.0,137.0,634.0,Tuesday,2021,9,8,JiuZhaigou,3912.603702,1840.125736,3975.270562
526,2021-09-09,4295.0,527,439.0,986.0,146.0,652.0,Wednesday,2021,9,9,JiuZhaigou,4435.783450,2100.821748,3758.394802
527,2021-09-10,5056.0,528,426.0,1077.0,137.0,586.0,Thursday,2021,9,10,JiuZhaigou,4964.977175,2096.338414,3994.684411
528,2021-09-11,5754.0,529,265.0,1206.0,146.0,889.0,Friday,2021,9,11,JiuZhaigou,5500.962899,2200.653459,4052.383642


In [18]:
# Trend

In [51]:
pretrained_model_paths=['.\\saved_models\\best_model_epoch=34-v48.ckpt', '.\\saved_models\\best_model_epoch=21-v7.ckpt', '.\\saved_models\\best_model_epoch=26-v34.ckpt', '.\\saved_models\\best_model_epoch=26-v34.ckpt', '.\\saved_models\\best_model_epoch=30-v15.ckpt', '.\\saved_models\\best_model_epoch=24-v36.ckpt', '.\\saved_models\\best_model_epoch=34-v48.ckpt', '.\\saved_models\\best_model_epoch=38-v87.ckpt', '.\\saved_models\\best_model_epoch=24-v34.ckpt', '.\\saved_models\\best_model_epoch=32-v26.ckpt', '.\\saved_models\\best_model_epoch=36-v65.ckpt', '.\\saved_models\\best_model_epoch=36-v64.ckpt', '.\\saved_models\\best_model_epoch=32-v26.ckpt', '.\\saved_models\\best_model_epoch=26-v34.ckpt', '.\\saved_models\\best_model_epoch=24-v34.ckpt', '.\\saved_models\\best_model_epoch=30-v15.ckpt', '.\\saved_models\\best_model_epoch=39-v71.ckpt', '.\\saved_models\\best_model_epoch=38-v84.ckpt', '.\\saved_models\\best_model_epoch=39-v73.ckpt', '.\\saved_models\\best_model_epoch=32-v26.ckpt', '.\\saved_models\\best_model_epoch=26-v34.ckpt', '.\\saved_models\\best_model_epoch=37-v31.ckpt', '.\\saved_models\\best_model_epoch=38-v84.ckpt', '.\\saved_models\\best_model_epoch=21-v7.ckpt', '.\\saved_models\\best_model_epoch=34-v48.ckpt', '.\\saved_models\\best_model_epoch=37-v31.ckpt', '.\\saved_models\\best_model_epoch=38-v86.ckpt', '.\\saved_models\\best_model_epoch=38-v86.ckpt', '.\\saved_models\\best_model_epoch=24-v35.ckpt', '.\\saved_models\\best_model_epoch=38-v84.ckpt']

In [52]:

start_prediction_idx = 441
end_prediction_idx = 530  # Includes 90 time points
prediction_length = 3  # The step length for each prediction
max_encoder_length=30

# Initialize an empty list to store all predictions
all_predictions = []



In [53]:
# Used for creating the validation set
training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= 440], 
        time_idx="time_idx",
        target="Trend",
        min_encoder_length=30 // 2,
        max_encoder_length=30,
        min_prediction_length=1,
        max_prediction_length=3,
        time_varying_known_categoricals=["month","weekday","day"],
        time_varying_known_reals=["time_idx","pc_Jiuzhaigou","mob_Jiuzhaigou","pc_SichuanEpidemic","mob_SichuanEpidemic"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=[
            "Trend",

        ],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(
            groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,)

In [54]:
# Loop for making predictions
for i, model_path in enumerate(pretrained_model_paths):
    start_idx = start_prediction_idx + i * prediction_length
    end_idx = start_idx + prediction_length

    if end_idx > end_prediction_idx:
        end_idx = end_prediction_idx + 1  
    
    # Create a new validation dataset
    validation_data = data[data['time_idx'] < end_idx].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    # Load the pretrained model and make predictions
    best_tft = TemporalFusionTransformer.load_from_checkpoint(model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    current_prediction = raw_predictions[0][:, :, 3]   # Retrieve current prediction results
    current_prediction_numpy = current_prediction.numpy().flatten()
    all_predictions.extend(current_prediction_numpy.tolist())

# Print all predictions
print(all_predictions)

[6129.64453125, 6352.8642578125, 6447.2138671875, 7830.84375, 7406.046875, 7321.92626953125, 7666.904296875, 7698.93359375, 7833.60107421875, 7484.5986328125, 7752.76953125, 7740.767578125, 8605.42578125, 8817.4208984375, 8854.2861328125, 9757.916015625, 10233.0224609375, 10970.4111328125, 12144.359375, 12906.7587890625, 13364.46875, 14181.515625, 14419.916015625, 14807.205078125, 15359.283203125, 15862.1171875, 16507.197265625, 16982.05859375, 17298.01171875, 17015.29296875, 16853.220703125, 16347.4296875, 16248.537109375, 17291.73046875, 17608.064453125, 17539.021484375, 17894.3984375, 17827.96875, 17802.23046875, 17299.494140625, 17094.75, 16998.712890625, 15096.970703125, 15631.85546875, 15578.88671875, 8383.38671875, 7688.65087890625, 6498.47509765625, 2722.222900390625, 2574.366455078125, 2569.16259765625, 1749.000732421875, 1829.287109375, 1591.991943359375, 857.2880859375, 841.65283203125, 692.52880859375, 3248.25927734375, 3017.31396484375, 2929.077392578125, 1577.631103515625

In [1]:
Trend_fore=[6129.64453125, 6352.8642578125, 6447.2138671875, 7830.84375, 7406.046875, 7321.92626953125, 7666.904296875, 7698.93359375, 7833.60107421875, 7484.5986328125, 7752.76953125, 7740.767578125, 8605.42578125, 8817.4208984375, 8854.2861328125, 9757.916015625, 10233.0224609375, 10970.4111328125, 12144.359375, 12906.7587890625, 13364.46875, 14181.515625, 14419.916015625, 14807.205078125, 15359.283203125, 15862.1171875, 16507.197265625, 16982.05859375, 17298.01171875, 17015.29296875, 16853.220703125, 16347.4296875, 16248.537109375, 17291.73046875, 17608.064453125, 17539.021484375, 17894.3984375, 17827.96875, 17802.23046875, 17299.494140625, 17094.75, 16998.712890625, 15096.970703125, 15631.85546875, 15578.88671875, 8383.38671875, 7688.65087890625, 6498.47509765625, 2722.222900390625, 2574.366455078125, 2569.16259765625, 1749.000732421875, 1829.287109375, 1591.991943359375, 857.2880859375, 841.65283203125, 692.52880859375, 3248.25927734375, 3017.31396484375, 2929.077392578125, 1577.631103515625, 1631.328857421875, 1501.458251953125, 1641.202392578125, 1478.2890625, 1330.93603515625, 1455.498046875, 1423.043701171875, 1394.558349609375, 1525.096923828125, 1401.773681640625, 1388.703125, 1227.0244140625, 1254.371826171875, 1287.27197265625, 1298.07421875, 1316.174560546875, 1186.3779296875, 1420.00048828125, 1895.378662109375, 1697.482666015625, 2449.923828125, 2519.457275390625, 2503.656494140625, 3113.26416015625, 3759.894775390625, 4236.86376953125, 5528.30517578125, 5969.59033203125, 6560.515625]


In [56]:
# Seasonal

In [59]:
pretrained_model_paths=['.\\saved_models\\best_model_epoch=7-v49.ckpt', '.\\saved_models\\best_model_epoch=22-v31.ckpt', '.\\saved_models\\best_model_epoch=38-v97.ckpt', '.\\saved_models\\best_model_epoch=30-v22.ckpt', '.\\saved_models\\best_model_epoch=12-v45.ckpt', '.\\saved_models\\best_model_epoch=13-v23.ckpt', '.\\saved_models\\best_model_epoch=8-v64.ckpt', '.\\saved_models\\best_model_epoch=8-v64.ckpt', '.\\saved_models\\best_model_epoch=8-v64.ckpt', '.\\saved_models\\best_model_epoch=7-v49.ckpt', '.\\saved_models\\best_model_epoch=3-v64.ckpt', '.\\saved_models\\best_model_epoch=7-v49.ckpt', '.\\saved_models\\best_model_epoch=11-v19.ckpt', '.\\saved_models\\best_model_epoch=7-v49.ckpt', '.\\saved_models\\best_model_epoch=7-v49.ckpt', '.\\saved_models\\best_model_epoch=11-v20.ckpt', '.\\saved_models\\best_model_epoch=8-v64.ckpt', '.\\saved_models\\best_model_epoch=19-v16.ckpt', '.\\saved_models\\best_model_epoch=4-v70.ckpt', '.\\saved_models\\best_model_epoch=6-v74.ckpt', '.\\saved_models\\best_model_epoch=18-v31.ckpt', '.\\saved_models\\best_model_epoch=7-v49.ckpt', '.\\saved_models\\best_model_epoch=7-v49.ckpt', '.\\saved_models\\best_model_epoch=25-v18.ckpt', '.\\saved_models\\best_model_epoch=31-v14.ckpt', '.\\saved_models\\best_model_epoch=36-v72.ckpt', '.\\saved_models\\best_model_epoch=34-v50.ckpt', '.\\saved_models\\best_model_epoch=7-v49.ckpt', '.\\saved_models\\best_model_epoch=7-v49.ckpt', '.\\saved_models\\best_model_epoch=8-v64.ckpt']


In [60]:
training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= 440],  
        time_idx="time_idx",
        target="Seasonal",
        min_encoder_length=30 // 2,
        max_encoder_length=30,
        min_prediction_length=1,
        max_prediction_length=3,
        time_varying_known_categoricals=["month","weekday","day"],
        time_varying_known_reals=["time_idx","pc_Jiuzhaigou","mob_Jiuzhaigou","pc_SichuanEpidemic","mob_SichuanEpidemic"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=[
            "Seasonal",

        ],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(
            groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,)

In [61]:
all_predictions = []
for i, model_path in enumerate(pretrained_model_paths):
    start_idx = start_prediction_idx + i * prediction_length
    end_idx = start_idx + prediction_length

    if end_idx > end_prediction_idx:
        end_idx = end_prediction_idx + 1 
    
    validation_data = data[data['time_idx'] < end_idx].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    best_tft = TemporalFusionTransformer.load_from_checkpoint(model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    current_prediction = raw_predictions[0][:, :, 3] 
    current_prediction_numpy = current_prediction.numpy().flatten()
    all_predictions.extend(current_prediction_numpy.tolist())

print(all_predictions)

[1944.33642578125, 1950.6783447265625, 1945.698974609375, 2080.982177734375, 2629.7373046875, 1889.21923828125, 1770.9111328125, 1719.2886962890625, 1975.5142822265625, 1831.391357421875, 2087.447509765625, 2375.411376953125, 1785.48681640625, 1777.363037109375, 1744.605712890625, 2078.33642578125, 1860.61181640625, 1935.3638916015625, 2246.82568359375, 2173.355712890625, 1723.951416015625, 1718.109130859375, 2042.7100830078125, 1791.0909423828125, 1999.5714111328125, 2172.775634765625, 2122.702880859375, 1944.25, 1917.136962890625, 1930.862060546875, 1767.88818359375, 1772.8466796875, 1739.6881103515625, 1984.2308349609375, 1965.3115234375, 1921.02978515625, 2022.0223388671875, 1833.4239501953125, 2047.3070068359375, 2041.8695068359375, 2049.154296875, 2030.793212890625, 1906.641357421875, 2001.7928466796875, 2001.112548828125, 1986.478759765625, 1996.638916015625, 1892.606689453125, 1871.9881591796875, 1855.957763671875, 1942.00830078125, 1900.54443359375, 1931.8167724609375, 2049.40

In [2]:
Seasonal_fore=[1944.33642578125, 1950.6783447265625, 1945.698974609375, 2080.982177734375, 2629.7373046875, 1889.21923828125, 1770.9111328125, 1719.2886962890625, 1975.5142822265625, 1831.391357421875, 2087.447509765625, 2375.411376953125, 1785.48681640625, 1777.363037109375, 1744.605712890625, 2078.33642578125, 1860.61181640625, 1935.3638916015625, 2246.82568359375, 2173.355712890625, 1723.951416015625, 1718.109130859375, 2042.7100830078125, 1791.0909423828125, 1999.5714111328125, 2172.775634765625, 2122.702880859375, 1944.25, 1917.136962890625, 1930.862060546875, 1767.88818359375, 1772.8466796875, 1739.6881103515625, 1984.2308349609375, 1965.3115234375, 1921.02978515625, 2022.0223388671875, 1833.4239501953125, 2047.3070068359375, 2041.8695068359375, 2049.154296875, 2030.793212890625, 1906.641357421875, 2001.7928466796875, 2001.112548828125, 1986.478759765625, 1996.638916015625, 1892.606689453125, 1871.9881591796875, 1855.957763671875, 1942.00830078125, 1900.54443359375, 1931.8167724609375, 2049.40185546875, 1955.144287109375, 1918.985595703125, 2066.11328125, 1986.6361083984375, 1999.9002685546875, 2014.7960205078125, 2200.968017578125, 2058.9970703125, 1922.5321044921875, 1962.42919921875, 1950.079833984375, 1960.69970703125, 1990.1719970703125, 1965.51171875, 1961.332763671875, 1926.1170654296875, 1951.14013671875, 1970.833251953125, 1980.037109375, 2003.146240234375, 2092.108642578125, 2035.389404296875, 1913.3680419921875, 1949.2423095703125, 1993.648681640625, 1949.2208251953125, 1979.8212890625, 2032.2301025390625, 2006.8978271484375, 1951.727294921875, 2009.889404296875, 1982.73388671875, 1970.2703857421875, 2015.40185546875, 2239.109619140625, 2157.64501953125]

In [63]:
# Resid

In [66]:
pretrained_model_paths=['.\\saved_models\\best_model_epoch=2-v55.ckpt', '.\\saved_models\\best_model_epoch=10-v53.ckpt', '.\\saved_models\\best_model_epoch=27-v15.ckpt', '.\\saved_models\\best_model_epoch=7-v50.ckpt', '.\\saved_models\\best_model_epoch=36-v75.ckpt', '.\\saved_models\\best_model_epoch=37-v33.ckpt', '.\\saved_models\\best_model_epoch=10-v53.ckpt', '.\\saved_models\\best_model_epoch=36-v75.ckpt', '.\\saved_models\\best_model_epoch=36-v75.ckpt', '.\\saved_models\\best_model_epoch=2-v57.ckpt', '.\\saved_models\\best_model_epoch=2-v56.ckpt', '.\\saved_models\\best_model_epoch=36-v75.ckpt', '.\\saved_models\\best_model_epoch=2-v57.ckpt', '.\\saved_models\\best_model_epoch=18-v32.ckpt', '.\\saved_models\\best_model_epoch=20-v25.ckpt', '.\\saved_models\\best_model_epoch=2-v55.ckpt', '.\\saved_models\\best_model_epoch=2-v57.ckpt', '.\\saved_models\\best_model_epoch=6-v75.ckpt', '.\\saved_models\\best_model_epoch=10-v54.ckpt', '.\\saved_models\\best_model_epoch=36-v74.ckpt', '.\\saved_models\\best_model_epoch=36-v75.ckpt', '.\\saved_models\\best_model_epoch=31-v15.ckpt', '.\\saved_models\\best_model_epoch=38-v91.ckpt', '.\\saved_models\\best_model_epoch=34-v51.ckpt', '.\\saved_models\\best_model_epoch=10-v53.ckpt', '.\\saved_models\\best_model_epoch=22-v26.ckpt', '.\\saved_models\\best_model_epoch=36-v75.ckpt', '.\\saved_models\\best_model_epoch=36-v75.ckpt', '.\\saved_models\\best_model_epoch=10-v53.ckpt', '.\\saved_models\\best_model_epoch=2-v57.ckpt']

In [67]:
training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= 440], 
        time_idx="time_idx",
        target="Resid",
        min_encoder_length=30 // 2,
        max_encoder_length=30,
        min_prediction_length=1,
        max_prediction_length=3,
        time_varying_known_categoricals=["month","weekday","day"],
        time_varying_known_reals=["time_idx","pc_Jiuzhaigou","mob_Jiuzhaigou","pc_SichuanEpidemic","mob_SichuanEpidemic"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=[
            "Resid",

        ],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(
            groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,)

In [68]:
all_predictions = []
for i, model_path in enumerate(pretrained_model_paths):
    start_idx = start_prediction_idx + i * prediction_length
    end_idx = start_idx + prediction_length

    if end_idx > end_prediction_idx:
        end_idx = end_prediction_idx + 1 
    
    validation_data = data[data['time_idx'] < end_idx].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    best_tft = TemporalFusionTransformer.load_from_checkpoint(model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    current_prediction = raw_predictions[0][:, :, 3]  
    current_prediction_numpy = current_prediction.numpy().flatten()
    all_predictions.extend(current_prediction_numpy.tolist())

print(all_predictions)

[3199.1083984375, 3408.634765625, 3330.770751953125, 3978.34228515625, 4089.282470703125, 4006.98046875, 4108.45556640625, 4074.5234375, 4093.560302734375, 3902.927490234375, 3882.847412109375, 3930.942626953125, 3786.60302734375, 3858.351806640625, 3790.992431640625, 3726.561279296875, 3683.5224609375, 3897.167236328125, 4036.590087890625, 3965.044189453125, 4018.591552734375, 3947.942138671875, 3934.564697265625, 3919.7373046875, 3971.0419921875, 4042.477294921875, 4097.01220703125, 4620.3916015625, 4615.5673828125, 4549.8330078125, 3614.728759765625, 3228.113525390625, 3277.65283203125, 3290.057861328125, 3965.40771484375, 4159.5537109375, 4822.0224609375, 4679.08056640625, 4696.54296875, 4465.34033203125, 4559.55029296875, 4416.97802734375, 4640.41015625, 4748.53564453125, 4736.08544921875, 2498.828125, 2790.7314453125, 2911.620361328125, 3552.962646484375, 3365.640625, 2872.783203125, 3405.2783203125, 3290.61279296875, 3512.73974609375, 4050.147705078125, 4069.25341796875, 4080.82

In [3]:
Resid_fore=[3199.1083984375, 3408.634765625, 3330.770751953125, 3978.34228515625, 4089.282470703125, 4006.98046875, 4108.45556640625, 4074.5234375, 4093.560302734375, 3902.927490234375, 3882.847412109375, 3930.942626953125, 3786.60302734375, 3858.351806640625, 3790.992431640625, 3726.561279296875, 3683.5224609375, 3897.167236328125, 4036.590087890625, 3965.044189453125, 4018.591552734375, 3947.942138671875, 3934.564697265625, 3919.7373046875, 3971.0419921875, 4042.477294921875, 4097.01220703125, 4620.3916015625, 4615.5673828125, 4549.8330078125, 3614.728759765625, 3228.113525390625, 3277.65283203125, 3290.057861328125, 3965.40771484375, 4159.5537109375, 4822.0224609375, 4679.08056640625, 4696.54296875, 4465.34033203125, 4559.55029296875, 4416.97802734375, 4640.41015625, 4748.53564453125, 4736.08544921875, 2498.828125, 2790.7314453125, 2911.620361328125, 3552.962646484375, 3365.640625, 2872.783203125, 3405.2783203125, 3290.61279296875, 3512.73974609375, 4050.147705078125, 4069.25341796875, 4080.8251953125, 944.080810546875, 1254.1982421875, 1799.219482421875, 3911.884033203125, 3943.30712890625, 3915.33837890625, 4048.28857421875, 4048.4443359375, 4056.927978515625, 3983.52587890625, 4030.406494140625, 3982.89404296875, 4060.75146484375, 4013.85986328125, 3932.33544921875, 4139.5380859375, 4043.35107421875, 4019.3564453125, 3887.363037109375, 3869.89453125, 3915.141845703125, 3731.5400390625, 3822.80810546875, 3785.784423828125, 3791.187255859375, 3800.80322265625, 3859.1884765625, 4106.90966796875, 4085.8076171875, 4086.601806640625, 3914.073974609375, 3582.1708984375, 3307.11328125]

In [4]:
true_values=[5186.0, 5796.0, 6488.0, 7816.0, 8132.0, 7208.0, 7661.0, 7348.0, 8117.0, 7213.0, 8231.0, 8095.0, 8201.0, 8540.0, 8322.0, 9564.0, 9779.0, 10797.0, 12469.0, 13659.0, 13053.0, 13844.0, 15020.0, 14585.0, 15329.0, 16125.0, 17022.0, 17647.0, 19139.0, 20154.0, 16296.0, 16429.0, 14781.0, 16233.0, 19551.0, 19045.0, 19782.0, 18719.0, 19424.0, 17786.0, 17561.0, 18200.0, 18379.0, 18592.0, 16315.0, 9602.0, 6386.0, 4111.0, 2819.0, 2027.0, 1691.0, 1112.0, 1156.0, 1156.0, 739.0, 800.0, 797.0, 824.0, 753.0, 971.0, 1214.0, 1508.0, 1425.0, 1814.0, 1575.0, 1579.0, 1440.0, 1410.0, 1508.0, 1570.0, 1422.0, 1436.0, 1357.0, 1258.0, 1348.0, 1269.0, 894.0, 1065.0, 1158.0, 1648.0, 1639.0, 2242.0, 2393.0, 2419.0, 3137.0, 3728.0, 4295.0, 5056.0, 5754.0, 6377.0]

In [6]:
import numpy as np
# Combine trend, season, and residual components to get the final 'tourist' forecast

tourist = [sum(x) for x in zip(Trend_fore, Seasonal_fore, Resid_fore)]
tourist = [x - 6000 for x in tourist]


# Calculate Mean Absolute Error (MAE), Root Mean Square Error (RMSE), and Mean Absolute Percentage Error (MAPE)
mae = np.mean(np.abs(np.array(true_values) - np.array(tourist)))
rmse = np.sqrt(np.mean(np.square(np.array(true_values) - np.array(tourist))))
mape = np.mean(np.abs(np.array(true_values) - np.array(tourist)) / np.array(true_values))

# Print the evaluation metrics
print("MAE:", mae)
print("RMSE:", rmse)
print("MAPE:", mape)

MAE: 355.01085611979164
RMSE: 700.6990225866793
MAPE: 0.07182501903118997
